# Prep

In [ ]:
import json
import numpy as np
import os
import pandas as pd
import plotnine as gg
gg.theme_set(gg.theme_classic)
default_figure_size = (6.4, 4.8)

In [ ]:
# data_dir = 'C:/Users/maria/MEGAsync/Berkeley/CHaRLy/data/mTurk1'
data_dir = 'C:/Users/maria/MEGAsync/Berkeley/CHaRLy/data/RPP34'
plot_dir = data_dir + 'figures'
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

In [ ]:
all_data = pd.read_csv(os.path.join(data_dir, 'all_data_raw.csv'), index_col=0)
all_data

In [ ]:
demographics = pd.read_csv(os.path.join(data_dir, 'Demographics.csv'), header=1)
demographics = demographics.rename(columns={'id': 'sid'})
demographics.sid = demographics.sid.astype('int')
demographics

In [ ]:
# Subset participants of current dataset
demo = demographics.loc[demographics['sid'].isin(np.unique(all_data['sid']))].reset_index(drop=True)
demo

In [ ]:
# Remove duplicate entries
already_ids = []

for i, sid in enumerate(demo['sid']):
    if sid in already_ids:
        print("Removing duplicate entry for participant {}.".format(sid))
        demo = demo.drop(i)
    already_ids += [sid]
demo = demo.reset_index(drop=True)
demo.shape

# Exclude Participants

In [ ]:
min_points_chance_performer = 25
max_missed_trials_inattentive = 40

## Psychological disorder, head trauma

In [ ]:
psych_sids = demo.loc[demo['mental/psychiatric illness diagnosis'] == 'Yes', 'sid'].values
trauma_sids = demo.loc[demo['head trauma/neurological disorder'] == 'Yes', 'sid'].values

psych_sids, trauma_sids

## Bad performance

In [ ]:
# Add chance performer column to all_data
id_cols = ['sid', 'phase', 'trial_type']
chance_performers1 = all_data.groupby(id_cols).aggregate('mean')['points'].reset_index()
chance_performers1 = chance_performers1.groupby(['sid']).aggregate('count')['phase'].reset_index()
chance_performers1

In [ ]:
chance_performers2 = all_data.groupby(id_cols).aggregate('mean').reset_index()[id_cols + ['acc']]
chance_performers2['points'] = chance_performers2['acc'] * (np.max(all_data.trial_) + 1)
chance_performers2

In [ ]:
chance_performers = set(
    list(chance_performers1.loc[chance_performers1['phase'] == 1, 'sid']) + \
    list(set(chance_performers2.loc[
        (chance_performers2.trial_type == 'learning') & 
        (chance_performers2.points < min_points_chance_performer), 'sid']))
)

print("Chance performer ids: {}".format(chance_performers))

## Too many missed trials

In [ ]:
d = all_data.loc[np.isnan(all_data['key_press'])].groupby(['sid']).aggregate('sum').reset_index()[['sid', 'timeout']]

inattentives = d.loc[d['timeout'] > max_missed_trials_inattentive].sid.values

print("Inattentive participant ids: {}".format(inattentives))

In [ ]:
g = (gg.ggplot(d, gg.aes('timeout'))
 + gg.geom_histogram(position=gg.position_dodge(width=4))
 + gg.geom_vline(xintercept=max_missed_trials_inattentive, linetype='dotted', color='red')
)
g.save(os.path.join(plot_dir, '01_ChancePerformers.png'))
print(g)

## Save data

In [ ]:
# Add columns to all_data
all_data['chance_performer'] = all_data['sid'].isin(chance_performers)
all_data['inattentives'] = all_data['sid'].isin(inattentives)
all_data['psych_disorder'] = all_data['sid'].isin(psych_sids)
all_data['head_trauma'] = all_data['sid'].isin(trauma_sids)

# Save to disk
all_data.to_csv(os.path.join(data_dir, 'all_data.csv'))

# Demographics

In [ ]:
print('Number of Females: {}; Males: {}; Decline to answer: {}'.format(
    np.sum(demo['sex'] == "Female"), np.sum(demo['sex'] == "Male"), np.sum(demo['sex'] == "Decline to answer")
))

In [ ]:
demo['age_years_months'] = demo['age (years)'] + demo['age (months)'] / 12

print('Mean age: {} (std: {})'.format(
    np.mean(demo['age_years_months']), np.std(demo['age_years_months'])
))

g = (gg.ggplot(demo, gg.aes('age_years_months'))
     + gg.geom_histogram()
    )
g.save(os.path.join(plot_dir, '01_ageHistogram.png'))
print(g)